In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Описание входных данных и цели проекта
#### Входные данные: 
12500 текстовых файлов положительных отзывов о фильмах и 12500 тысяч отрицательных отзывов. В названии каждого файла содержится итоговый балл за фильм, присвоенный отзывающимся.

#### Цель проекта: 
составить функцию отзыва, максимально коррелирующую с итоговым баллом отзыва.

#### Пояснения по ходу решения:
В процессе построения функции будет создан словарь и каждому слову словаря будет присвоена эмпирическая вероятность того, что отзыв, в котором находится это слово - положительный. По определенным параметрам выделяется срез словаря, значимый для задачи разделения. По каждому отзыву с учетом среза вычисляется средняя вероятность того, что отзыв положительный, и проводится оптимизация по множеству параметров среза (вручную) для достижения максимального коэффициента корреляции между баллом и средней вероятностью.
Оптимальная совокупность параметров позволяет достигнуть корреляции около r=0.86 при использовании менее 10% слов базового датафрейма

### Вспомогательные функции для парсинга

In [2]:
import re

def cleanhtml(raw_html):
    # убрать html-теги
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [3]:
def words(stringIterable):
    #разбивает файл (поток ввода) на слова
    lineStream = iter(stringIterable)
    for line in lineStream: #enumerate the lines
        line = cleanhtml(line)
        for word in line.split(): #further break them down
            if re.match("^[A-Za-z]+[-]?[A-Za-z0-9]+$", word):
                yield word

### Сборка базового датафрейма

In [4]:
# собирает в массив все слова из позитивных отзывов
import os
a = []
file_list = os.listdir(path='aclImdb/train/pos')
i = 0
for fl in file_list:
    ball = int(fl[fl.find('_')+1 : fl.find('.')])
    f = open('aclImdb/train/pos/' + fl, encoding="UTF-8")
    a += [[w.strip('!?,.:;()"').lower(), 1, 0, fl, ball] for w in words(f)]
    f.close()
    i +=1
    if i%3000 == 0:
        # индикатор процесса
        print(i)
    

3000
6000
9000
12000


In [5]:
# добирает в массив все слова из негативных отзывов
file_list = os.listdir(path='aclImdb/train/neg')
i = 0
for fl in file_list:
    ball = int(fl[fl.find('_')+1 : fl.find('.')])
    f = open('aclImdb/train/neg/' + fl, encoding="UTF-8")
    a += [[w.strip('!?,.:;()"').lower(), 0, 1, fl, ball] for w in words(f)]
    f.close()
    i +=1
    if i%3000 == 0:
        print(i)

3000
6000
9000
12000


In [6]:
df = pd.DataFrame(a, columns =['word', 'ispos', 'isneg', 'comm_file', 'score'])

In [7]:
df.shape

(4639666, 5)

In [8]:
df[['ispos', 'isneg']].sum()

ispos    2359412
isneg    2280254
dtype: int64

### Формирование словаря частотностей

In [9]:
vocab = df.groupby('word')['ispos', 'isneg'].sum().reset_index().rename(columns={'ispos':'sumpos', 'isneg':'sumneg'})

In [10]:
vocab[['sumpos', 'sumneg']].sum()

sumpos    2359412
sumneg    2280254
dtype: int64

In [11]:
# Эмпирическая условная вероятность того, что слово находится в позитивном отзыве
vocab['pos'] = vocab['sumpos']/(vocab['sumpos'] + vocab['sumneg'])
# dfg['neg'] = dfg['isneg']/(dfg['ispos'] + dfg['isneg'])

In [12]:
vocab.head(30)

,word,sumpos,sumneg,pos
0,a-3,0,1,0.000000
1,a-5,0,1,0.000000
2,a-b,1,0,1.000000
3,a-bomb,1,0,1.000000
4,a-budget,0,1,0.000000
5,a-bustle,0,1,0.000000
6,a-class,2,0,1.000000
7,a-ever,1,0,1.000000
8,a-flutter,0,1,0.000000
9,a-game,0,1,0.000000


### Срез словаря по существенным словам

In [13]:
# назначим параметры для выбора среза из словаря 
# (эта и последующие ячейки выполнялись несколько раз с разными значениям параметров):
gcnt = 0 # нижняя граница для общего количества встречаемости слова
sep = 1.98 # отношение частот для разделения на позитив и негатив

In [14]:
v_slice = vocab.loc[(vocab['sumpos'] + vocab['sumneg'])>gcnt]


In [15]:
# отметим базовые слова в срезе
v_slice['posbase'] = (v_slice['sumpos'] > v_slice['sumneg']*sep)
v_slice['negbase'] = (v_slice['sumpos']*sep < v_slice['sumneg'])

In [16]:
# размеры среза и количества базовых слов
v_slice.shape[0], v_slice.loc[v_slice['posbase']==True].shape[0], v_slice.loc[v_slice['negbase']==True].shape[0]

(73539, 29353, 27155)

In [17]:
v_slice = v_slice[(v_slice['posbase']==True) | (v_slice['negbase']==True)]

In [18]:
v_slice.to_csv('v_slice.csv', index=None)

### Вычисление веса отзывов

In [19]:
res = df.merge(v_slice, on=['word'], how='inner')

In [20]:
res.shape # для построения метрики используется менее 10% слов базового датафрейма:

(469423, 10)

In [21]:
res1 = res.groupby('comm_file').agg({'pos': [np.mean], 'score': [np.max]}).reset_index()
res1.shape

(24952, 3)

### Расчет итоговой корреляции

In [22]:
res1.corr()

,,pos,score
,,mean,amax
pos,mean,1.000000,0.857948
score,amax,0.857948,1.000000


#### В силу недостаточности памяти компьютера проверку работы функции на тесте проводил в отдельном jupiter ноутбуке (pr2.ipynb). Получен результат r=0.65, что свидетельствует об эффекте переобучения модели. Данная проблема может быть решена увеличением тренигового датасета